In [ ]:
import datetime
import itertools
import os
import time

import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import pathpatch_2d_to_3d, Poly3DCollection
import numpy as np
import pandas as pd
from scipy.special import roots_legendre
from scipy.spatial import ConvexHull
from tqdm.auto import tqdm

from dosipy.field import poynting
from dosipy.utils.dataloader import (load_antenna_el_properties,
                                     load_sphere_coords)
from dosipy.utils.derive import holoborodko
from dosipy.utils.integrate import elementwise_rectquad as equad
from dosipy.utils.viz import (fig_config, save_fig, set_axes_equal,
                              set_colorblind)

In [ ]:
print(f'platform: {jax.lib.xla_bridge.get_backend().platform}')

In [ ]:
# jax.config.update("jax_enable_x64", True)

In [ ]:
set_colorblind()
%config InlineBackend.figure_format = 'retina'

In [ ]:
# utils

def cart2sph(x, y, z):
    """Return spherical given Cartesain coordinates."""
    r = jnp.sqrt(x ** 2 + y ** 2 + z ** 2)
    theta = jnp.arccos(z / r)
    phi = jnp.arctan2(y, x)
    return r, theta, phi


def sph2cart(r, theta, phi):
    """Return Cartesian given Spherical coordinates."""
    x = r * jnp.cos(phi) * jnp.sin(theta)
    y = r * jnp.sin(phi) * jnp.sin(theta)
    z = r * jnp.cos(theta)
    return x, y, z

## Current distribution along dipole antenna operating at 10 GHz

In [ ]:
f = 10e9  # operating frequency of the antenna
antenna_data = load_antenna_el_properties(f)
Is = antenna_data.ireal.to_numpy() + antenna_data.iimag.to_numpy() * 1j
Is = jnp.asarray(Is)
xs = antenna_data.x.to_numpy()
xs = jnp.asarray(xs)

In [ ]:
fig_config(latex=True, text_size=13, scaler=1, line_width=2)
fig = plt.figure()
ax = fig.add_subplot()
ax.plot(xs * 1000, jnp.abs(Is) * 1000, 'k-', label=r'$|I(x)|$')
ax.plot(xs * 1000, Is.real * 1000, 'r--', label='$\\Re{[I(x)]}$')
ax.plot(xs * 1000, Is.imag * 1000, 'b-.', label='$\\Im{[I(x)]}$')
ax.set(xlabel=r'$x$ [mm]', ylabel=r'$I(x)$ [mA]',
       xticks=[xs.min().item() * 1000,
               (xs.min().item() + xs.max().item()) * 1000 / 2,
               xs.max().item() * 1000],
       xticklabels=['$-L/2$', '$0$', '$L/2$'])
ax.grid()
ax.legend()
fig.tight_layout();

#fname = os.path.join('figures', 'I_dist')
#save_fig(fig, fname=fname)

## EM fields across the surface of the human head model (free space condition)

In [ ]:
r_c = load_sphere_coords(2312)  # coordinates
d = -5 / 1000  # some fixed distance between the head model and the antenna
xs = xs - xs.max() / 2  # fix antenna position w.r.t. the model position
ys = jnp.zeros_like(xs) + r_c['y'].min() + d  # y-coordinates of the antenna
zs = jnp.zeros_like(xs)  # z-coordinates of the antenna
dx = xs[1] - xs[0]  # finite difference
Is_x = holoborodko(Is, dx)  # spatial distribution of the current gradients

In [ ]:
S = r_c.apply(
    lambda row: poynting(row['x'], row['y'], row['z'], xs, ys, zs, f, Is, Is_x),
    axis=1, result_type='expand')
S.columns = ['Sx', 'Sy', 'Sz']
S_abs = S.apply(
    lambda row: jnp.sqrt(row['Sx'].real ** 2 + row['Sy'].real ** 2 + row['Sz'].real ** 2),
    axis=1)
S.loc[:, 'S_abs'] = S_abs

# update dataframe with computed power density values
r_c_calc = pd.concat([r_c, S], axis=1)

In [ ]:
hull = ConvexHull(r_c_calc[['x', 'y', 'z']].to_numpy())
triangle_coords = hull.points[hull.simplices]

fig_config(latex=True, text_size=16, scaler=1.5)
fig = plt.figure()
ax = plt.axes(projection ='3d')
ax.add_collection3d(Poly3DCollection(triangle_coords, color='white'))
ax.scatter(r_c_calc['x'], r_c_calc['y'], r_c_calc['z'], s=10,
           c=r_c_calc['S_abs'], cmap='viridis')
ax.plot(xs, ys, zs, 'r-', zorder=4, lw=2,
        label=(f'antenna\n$d = {-int(d * 1000)}$ mm\n$f = {int(f / 1e9)}$ GHz'))
ax.set(xlabel='$x$ [m]', ylabel='$y$ [m]', zlabel='$z$ [m]',
       xticks=[-0.05, 0.0, 0.05],
       yticks=[-0.05, 0.0, 0.05],
       zticks=[-0.05, 0.0, 0.05],
       xlim3d=[-0.1, 0.1],
       ylim3d=[-0.1, 0.1],
       zlim3d=[-0.1, 0.1])
ax.tick_params(axis='z', which='major', pad=10)
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False
ax.xaxis.labelpad = 12
ax.yaxis.labelpad = 12
ax.zaxis.labelpad = 18

ax.view_init(20, -60)
ax = set_axes_equal(ax)
ax.set_box_aspect([1, 1, 1])
fig.legend(bbox_to_anchor=(0.875, 1.075))
fig.tight_layout();

#fname = os.path.join('figures', 'PD_dist_3D')
#save_fig(fig, fname=fname, formats=['png'])

In [ ]:
fig_config(latex=True, text_size=16, scaler=1.5)
fig = plt.figure(figsize=(3.25, 2.85))
ax = fig.add_subplot()
ax.axis('equal')
cs = ax.scatter(r_c_calc['x'], r_c_calc['z'], s=7,
                c=r_c_calc['S_abs'], cmap='viridis')
ax.plot(xs, zs, 'r-', lw=2)
ax.yaxis.tick_right()
ax.yaxis.set_label_position("right")
ax.set_xlabel('$x$ [m]')
ax.set_ylabel('$z$ [m]')
ax.set_xticks([-0.05, 0.05])
ax.set_yticks([-0.05, 0.05])
fig.tight_layout();

#fname = os.path.join('figures', 'PD_dist_2D')
#save_fig(fig, fname=fname, formats=['png'])

In [ ]:
# define the averaging area for assessment of the incident power density

target_area = (2 / 100, 2 / 100)
A = target_area[0] * target_area[1]
target_area_origin = (-target_area[0]/2, -target_area[1]/2)

r_c_calc_ta = r_c_calc[
    (r_c_calc['y'] < 0) &
    (r_c_calc['x'] > target_area_origin[0]) &
    (r_c_calc['x'] < target_area_origin[0] * -1) &
    (r_c_calc['z'] > target_area_origin[1]) &
    (r_c_calc['z'] < target_area_origin[1] * -1)]
r_c_calc_ta.reset_index(drop=True, inplace=True)

xt = r_c_calc_ta['x'].to_numpy()
yt = r_c_calc_ta['y'].to_numpy()
zt = r_c_calc_ta['z'].to_numpy()

In [ ]:
# generate Gauss-Legnedre integration nodes in spherical c.s. and convert to Cartesian

_, theta, phi = cart2sph(xt, yt, zt)
theta_a = theta.min()
theta_b = theta.max()
phi_a = phi.min()
phi_b = phi.max()
roots, weights = roots_legendre(11)
theta_points = 0.5 * (roots + 1.) * (theta_b - theta_a) + theta_a
theta_weights = 0.5 * weights * (theta_b - theta_a)
phi_points = 0.5 * (roots + 1.) * (phi_b - phi_a) + phi_a
phi_weights = 0.5 * weights * (phi_b - phi_a)

r = 0.09  # radius of the head model
phi_grid, theta_grid = jnp.meshgrid(phi_points, theta_points)
xt_spherical, yt_spherical, zt_spherical = sph2cart(r, theta_grid.ravel(), phi_grid.ravel())

In [ ]:
# compute the EM power density at integration nodes

Sx, Sy, Sz = [], [], []
for _xt, _yt, _zt in zip(xt_spherical, yt_spherical, zt_spherical):
    _Sx, _Sy, _Sz = poynting(_xt, _yt, _zt, xs, ys, zs, f, Is, Is_x)
    Sx.append(_Sx)
    Sy.append(_Sy)
    Sz.append(_Sz)
Sx = jnp.asarray(Sx)
Sy = jnp.asarray(Sy)
Sz = jnp.asarray(Sz)
Sr = 1/2 * np.sqrt(Sx.real ** 2 + Sy.real ** 2 + Sz.real ** 2)

In [ ]:
# visualize absolute values of the power density at integration nodes

fig_config(latex=True, text_size=16, scaler=1.5)

fig = plt.figure()
ax = plt.axes(projection ='3d')
cs = ax.scatter(xt_spherical, yt_spherical, zt_spherical, s=30,
                c=Sr, cmap='viridis')
cbar = fig.colorbar(cs, pad=0.2)
cbar.ax.set_ylabel(r'$|\boldsymbol{S}|$ [W/m$^2$]')
ax.plot(xs, ys, zs, linestyle='-', c='r')
ax.set(xlabel='$x$ [m]', ylabel='$y$ [m]', zlabel='$z$ [m]',
       xticks=[-0.005, 0.0, 0.005],
       yticks=[-0.093, -0.091, -0.089],
       zticks=[-0.005, 0.0, 0.005])
ax.tick_params(axis='z', which='major', pad=10)
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False
ax.xaxis.labelpad = 15
ax.yaxis.labelpad = 15
ax.zaxis.labelpad = 17
ax.view_init(25, -50)
fig.tight_layout();

#fname = os.path.join('figures', 'PD_dist_3D_avg_mag')
#save_fig(fig, fname=fname, formats=['png'])

In [ ]:
# visualize the power density vector field distribution across the averaging surface

fig_config(latex=True, text_size=16, scaler=1.5)
fig = plt.figure()
ax = plt.axes(projection ='3d')
ax.scatter(xt_spherical, yt_spherical, zt_spherical, s=40,
           ec='black', fc='None')
ax.plot(xs, ys, zs, linestyle='-', c='r', label='antenna')
q = ax.quiver(xt_spherical, yt_spherical, zt_spherical,
              jnp.abs(Sx), jnp.abs(Sy), jnp.abs(Sz),
              linewidths=1, length=0.002, normalize=True, color='b',
              label='$\\boldsymbol{S}(x,y,z)$')
ax.set(xlabel='$x$ [m]', ylabel='$y$ [m]', zlabel='$z$ [m]',
       xticks=[-0.005, 0.0, 0.005],
       yticks=[-0.093, -0.091, -0.089],
       zticks=[-0.005, 0.0, 0.005])
ax.tick_params(axis='z', which='major', pad=10)
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False
ax.xaxis.labelpad = 15
ax.yaxis.labelpad = 15
ax.zaxis.labelpad = 20
ax.view_init(25, -40)
ax.legend()
fig.tight_layout();

#fname = os.path.join('figures', 'PD_dist_3D_avg_vec')
#save_fig(fig, fname=fname, formats=['png'])

In [ ]:
# compute the incident power density (normal component), sPDn

nx, ny, nz = [], [], []
mag = 0
A_spherical = 0
for _theta, _w_theta in zip(theta_points, theta_weights):
    for _phi, _w_phi in zip(phi_points, phi_weights):
        # normal vector components
        _nx = r ** 2 * jnp.cos(_phi) * jnp.sin(_theta) ** 2 
        _ny = r ** 2 * jnp.sin(_phi) * jnp.sin(_theta) ** 2
        _nz = r ** 2 * jnp.cos(_theta) * jnp.sin(_theta)
        nx.append(_nx)
        ny.append(_ny)
        nz.append(_nz)
        # power density vector field
        _xt, _yt, _zt = sph2cart(r, _theta, _phi)
        _Sx, _Sy, _Sz = poynting(_xt, _yt, _zt, xs, ys, zs, f, Is, Is_x)
        # dot product between power density and normal vector
        _Sn = _Sx.real * _nx + _Sy.real * _ny + _Sz.real * _nz
        # incident power integration
        mag += _Sn * _w_theta * _w_phi
        # surface area integration
        A_spherical += jnp.sin(_theta) * r ** 2 * _w_theta * _w_phi
sPDn_spherical = mag / (2 * A_spherical)

print(f'spherical surface, sPDn = {sPDn_spherical:.4f} W/m2')

In [ ]:
# visualize the unit vector field distribution normal to the averaging surface

fig_config(latex=True, text_size=16, scaler=1.5)
fig = plt.figure()
ax = plt.axes(projection ='3d')
ax.scatter(xt_spherical, yt_spherical, zt_spherical, s=10,
           ec='black', fc='black')
antenna, = ax.plot(xs, ys, zs, linestyle='-', c='r', label='antenna')
z_ref = ax.get_zlim()[0]
ax.plot(xs, ys, 'r-', lw=1, zdir='z', zs=z_ref, alpha=0.5)
ax.plot(ys, np.linspace(z_ref, zs[0], ys.size), 'r--', lw=1, zdir='x', zs=xs.min(), alpha=0.5)
ax.plot(ys, np.linspace(z_ref, zs[0], ys.size), 'r--', lw=1, zdir='x', zs=xs.max(), alpha=0.5)
ax.quiver(xt_spherical, yt_spherical, zt_spherical,
          nx, ny, nz, arrow_length_ratio=0.25, 
          linewidths=1, length=0.0017, normalize=True, color='k')
arrow = ax.scatter([], [], [], c='k', marker='$\\longrightarrow$', s=1000, label='$\\boldsymbol{n}(x,y,z)$')
ax.set(xlabel='$x$ [mm]', ylabel='$y$ [mm]', zlabel='$z$ [mm]',
       xticks=[-0.01, 0.0, 0.01],
       yticks=[-0.095, -0.0925, -0.09],
       zticks=[-0.01, 0, 0.01],
       xticklabels=[-10, 0, 10],
       yticklabels=[-95, 92.5, -90],
       zticklabels=[-10, 0, 10],)
ax.tick_params(axis='z', which='major', pad=5)
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False
ax.xaxis.labelpad = 10
ax.yaxis.labelpad = 10
ax.zaxis.labelpad = 10
ax.view_init(25, -50)
fig.legend(handles=[antenna, arrow], bbox_to_anchor=(0.9, 1))
fig.tight_layout();

#fname = os.path.join('figures', 'n_dist_3D_avg')
#save_fig(fig, fname=fname)

In [ ]:
# visualize the normal component of the power density at integration nodes

n_norm = jnp.sqrt(jnp.asarray(nx) ** 2 + jnp.asarray(ny) ** 2 + jnp.asarray(nz) ** 2)
nx_normalized = jnp.asarray(nx) / n_norm
ny_normalized = jnp.asarray(ny) / n_norm
nz_normalized = jnp.asarray(nz) / n_norm
Sn = 1/2 * (Sx.real * nx_normalized + Sy.real * ny_normalized + Sz.real * nz_normalized)
fig_config(latex=True, text_size=16, scaler=1.5)

fig = plt.figure()
ax = plt.axes(projection ='3d')
cs = ax.scatter(xt_spherical, yt_spherical, zt_spherical, s=15,
                c=Sn, cmap='viridis')
cbar = fig.colorbar(cs, pad=0.15)
cbar.ax.set_ylabel(r'$\boldsymbol{S} \cdot \boldsymbol{n}$ [W/m$^2$]')
ax.plot(xs, ys, zs, linestyle='-', c='r')
z_ref = ax.get_zlim()[0]
ax.plot(xs, ys, 'r-', lw=1, zdir='z', zs=z_ref, alpha=0.5)
ax.plot(ys, np.linspace(z_ref, zs[0], ys.size), 'r--', lw=1, zdir='x', zs=xs.min(), alpha=0.5)
ax.plot(ys, np.linspace(z_ref, zs[0], ys.size), 'r--', lw=1, zdir='x', zs=xs.max(), alpha=0.5)
ax.set(xlabel='$x$ [mm]', ylabel='$y$ [mm]', zlabel='$z$ [mm]',
       xticks=[-0.01, 0.0, 0.01],
       yticks=[-0.095, -0.0925, -0.09],
       zticks=[-0.01, 0, 0.01],
       xticklabels=[-10, 0, 10],
       yticklabels=[-95, 92.5, -90],
       zticklabels=[-10, 0, 10],)
ax.tick_params(axis='z', which='major', pad=5)
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False
ax.xaxis.labelpad = 12
ax.yaxis.labelpad = 12
ax.zaxis.labelpad = 8
ax.view_init(25, -50)
fig.tight_layout();

#fname = os.path.join('figures', 'PD_dist_3D_avg_n')
#save_fig(fig, fname=fname, formats=['png'])

## EM exposure of planar tissue models to dipole antenna at mmWaves

In [ ]:
# generate coordinates of 3 different planar models for the averaging surface

_, theta, phi = cart2sph(xt, yt, zt)
theta_a = theta.min()
theta_b = theta.max()
phi_a = phi.min()
phi_b = phi.max()
roots, weights = roots_legendre(33)
theta_points = 0.5 * (roots + 1.) * (theta_b - theta_a) + theta_a
theta_weights = 0.5 * weights * (theta_b - theta_a)
phi_points = 0.5 * (roots + 1.) * (phi_b - phi_a) + phi_a
phi_weights = 0.5 * weights * (phi_b - phi_a)

r = 0.09  # radius of the head model
phi_grid, theta_grid = jnp.meshgrid(phi_points, theta_points)
xt_spherical, yt_spherical, zt_spherical = sph2cart(r, theta_grid.ravel(), phi_grid.ravel())
yt_planar_near = yt_spherical.min()
yt_planar_far = yt_spherical.max()
yt_planar_mid = (yt_planar_far + yt_planar_near) / 2

xt_planar = jnp.linspace(-target_area[0]/2, target_area[0]/2, 11)
zt_planar = jnp.linspace(-target_area[1]/2, target_area[1]/2, 11)

In [ ]:
# reconstruct the averaging surface

import open3d as o3d
pcd = o3d.geometry.PointCloud()
xyz = np.c_[xt_spherical, yt_spherical, zt_spherical]
pcd.points = o3d.utility.Vector3dVector(xyz)
pcd.paint_uniform_color([0.5, 0.5, 0.5])
pcd.estimate_normals()
pcd.orient_normals_consistent_tangent_plane(k=10)
radii = [0.005, 0.001, 0.002, 0.004]
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    pcd, o3d.utility.DoubleVector(radii))
triangle_coords = np.asarray(mesh.vertices)[np.asarray(mesh.triangles)]

In [ ]:
# visualize planar models relative to the spherical one

fig_config(latex=True, scaler=1, text_size=21)
fig = plt.figure()
ax = plt.axes(projection ='3d')

ax.plot(xs, ys, zs, linestyle='-', c='r', label='antenna')
z_ref = -0.01 * 1.1
ax.plot(xs, ys, 'r-', lw=1, zdir='z', zs=z_ref, alpha=0.5)
ax.plot(ys, np.linspace(z_ref, zs[0], ys.size), 'r--', lw=1, zdir='x', zs=xs.min(), alpha=0.5)
ax.plot(ys, np.linspace(z_ref, zs[0], ys.size), 'r--', lw=1, zdir='x', zs=xs.max(), alpha=0.5)

triangles = Poly3DCollection(triangle_coords, ec='None', fc='gray', alpha=0.5)
sur_planar_near = Rectangle(target_area_origin, target_area[0], target_area[1],
                  ec='r', ls='-', fc='None', lw=3, label='near')
sur_planar_mid = Rectangle(target_area_origin, target_area[0], target_area[1],
                  ec='b', ls='--', fc='None', lw=3, label='mid')
sur_planar_far = Rectangle(target_area_origin, target_area[0], target_area[1],
                  ec='g', ls='-.', fc='None', lw=3, label='far')
ax.add_patch(sur_planar_near)
ax.add_patch(sur_planar_mid)
ax.add_patch(sur_planar_far)
pathpatch_2d_to_3d(sur_planar_near, z=yt_planar_near, zdir='y')
pathpatch_2d_to_3d(sur_planar_mid, z=yt_planar_mid, zdir='y')
pathpatch_2d_to_3d(sur_planar_far, z=yt_planar_far, zdir='y')
ax.add_collection3d(triangles)

ax.set(xlabel='$x$ [mm]', ylabel='$y$ [mm]',
       xlim=[-0.01 * 1.1, 0.01 * 1.1],
       ylim=[-90.075 / 1000, -88.925 / 1000],
       zlim=[-0.01 * 1.1, 0.01 * 1.1],
       xticks=[-0.01, 0.0, 0.01],
       xticklabels=[-10, 0, 10],
       yticks=[-89/1000, -92/1000, -95/1000],
       yticklabels=[-89, -92, -95],
       zticks=[0.01, 0.0, 0.01], zticklabels=[])
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False
ax.xaxis.labelpad = 10
ax.yaxis.labelpad = 15
ax.zaxis.labelpad = 8
ax.view_init(22, -55)
box = ax.get_position()
fig.legend(ncol=4, bbox_to_anchor=(1.7, -0.05))
fig.tight_layout();

#fname = os.path.join('figures', 'surfaces_a')
#save_fig(fig, fname=fname, formats=['png'])

In [ ]:
fig_config(latex=True, scaler=1, text_size=21)
fig = plt.figure()
ax = plt.axes(projection ='3d')

triangles = Poly3DCollection(triangle_coords, ec='None', fc='gray', alpha=0.5)
sur_planar_near = Rectangle(target_area_origin, target_area[0], target_area[1],
                  ec='r', ls='-', fc='None', lw=3, label='near')
sur_planar_mid = Rectangle(target_area_origin, target_area[0], target_area[1],
                  ec='b', ls='--', fc='None', lw=3, label='mid')
sur_planar_far = Rectangle(target_area_origin, target_area[0], target_area[1],
                  ec='g', ls='-.', fc='None', lw=3, label='far')
ax.add_patch(sur_planar_near)
ax.add_patch(sur_planar_mid)
ax.add_patch(sur_planar_far)
pathpatch_2d_to_3d(sur_planar_near, z=yt_planar_near, zdir='y')
pathpatch_2d_to_3d(sur_planar_mid, z=yt_planar_mid, zdir='y')
pathpatch_2d_to_3d(sur_planar_far, z=yt_planar_far, zdir='y')
ax.add_collection3d(triangles)

ax.set(xlabel='$x$ [mm]', ylabel='$y$ [mm]', zlabel='$z$ [mm]',
       xlim=[-0.01 * 1.1, 0.01 * 1.1],
       ylim=[-90.075 / 1000, -88.925 / 1000],
       zlim=[-0.01 * 1.1, 0.01 * 1.1],
       xticks=[-0.01, 0.0, 0.01], xticklabels=[-10, 0, 10],
       yticks=[round(yt_planar_near, 4),
               round(yt_planar_mid, 4),
               round(yt_planar_far, 4)],
       yticklabels=[round(yt_planar_near * 1000),
                    round(yt_planar_mid * 1000, 1),
                    round(yt_planar_far * 1000)], 
       zticks=[-0.01, 0.0, 0.01], zticklabels=[-10, 0, 10])
ax.tick_params(axis='z', which='major', pad=5)
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False
ax.xaxis.labelpad = 10
ax.yaxis.labelpad = 15
ax.zaxis.labelpad = 8
ax.view_init(25, -50)
fig.tight_layout;

#fname = os.path.join('figures', 'surfaces_b')
#save_fig(fig, fname, formats=['png'])

In [ ]:
# incident power density (normal component) for planar model

def sPDn_planar_fn(yt_planar):
    S = np.empty((xt_planar.size, zt_planar.size))
    for xi, _xt in enumerate(xt_planar):
        for zi, _zt in enumerate(zt_planar):
            _Sx, _Sy, _Sz = poynting(_xt, yt_planar, _zt, xs, ys, zs, f, Is, Is_x)
            S[xi, zi] = -jnp.real(_Sy)  # only normal component is considered
    return 1 / (2 * A) * equad(xt_planar, zt_planar, S)


sPDn_planar_near = sPDn_planar_fn(yt_planar_near)
sPDn_planar_mid = sPDn_planar_fn(yt_planar_mid)
sPDn_planar_far = sPDn_planar_fn(yt_planar_far)

In [ ]:
print(f'spherical surface of A = {A_spherical * 1e4:.2f} cm2, IPD = {sPDn_spherical:.4f} W/m2')
print(f'near planar surface of A = {A * 1e4:.2f} cm2, IPD = {sPDn_planar_near:.4f} W/m2')
print(f'mid planar surface of A = {A * 1e4:.2f} cm2, IPD = {sPDn_planar_mid:.4f} W/m2')
print(f'far planar surface of A = {A * 1e4:.2f} cm2, IPD = {sPDn_planar_far:.4f} W/m2')